<a href="https://colab.research.google.com/github/saipragna25/dmasgn5/blob/main/dataminingA5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving glass.csv to glass.csv


In [2]:
import pandas as pd
dataset=pd.read_csv('glass.csv')
dataset.head()

,Id,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [3]:
!pip install faiss


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 


In [4]:
!sudo apt-get install libomp-dev


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 5 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (310 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debc

In [5]:
import pickle
import faiss
import numpy as np

In [6]:
data_vectors = dataset.drop(['Type of glass'], axis = 1)
data_vectors

,Id,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...
209,210,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
210,211,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
211,212,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
212,213,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [7]:
data_vectors = data_vectors.values
data_vectors = np.ascontiguousarray(data_vectors, dtype=np.float32)

In [8]:
data_labels = dataset['Type of glass']
data_labels.head()

0    1
1    1
2    1
3    1
4    1
Name: Type of glass, dtype: int64

In [9]:
data_labels = data_labels.values

Locality Sensitive Hashing

In [10]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors
        self.labels = labels    
   
    def build(self, num_bits=10):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]


In [11]:
lsh_index = LSHIndex(data_vectors, data_labels)
lsh_index.build()

In [13]:
lsh_index.query(np.array([data_vectors[213]]))

[7, 7, 7, 7, 7, 7, 1, 1, 1, 1]

In [14]:
lsh_index.query(np.array([data_vectors[0]]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Exhaustive Search

In [15]:
class ExhaustiveIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [16]:
exact_index = ExhaustiveIndex(data_vectors, data_labels)
exact_index.build()

In [19]:
exact_index.query(
  np.array([data_vectors[211]])
)

[7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

Product Quantization

In [20]:
class ProductQuantizationIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    
    def build(self, number_of_partition=1, search_in_x_partitions=1, subvector_size=2):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexI
        VFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [21]:
product_quantization_index = ProductQuantizationIndex(data_vectors, data_labels)
product_quantization_index.build()


In [23]:
product_quantization_index.query(np.array([data_vectors[200]]))

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

Trees and Graphs

In [24]:
!pip install annoy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 5.0 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=395185 sha256=9d04dafc6dc316bd731147b8c02172c3bd680b49476e78de5da2d065f50886b3
  Stored in directory: /root/.cache/pip/wheels/81/94/bf/92cb0e4fef8770fe9c6df0ba588fca30ab7c306b6048ae8a54
Successfully built annoy


In [25]:
import annoy

In [26]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors
        self.labels = labels    
   
    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimension)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k, search_k=7)                                           
        return [self.labels[i] for i in indices]

In [27]:
annoy_index = AnnoyIndex(data_vectors, data_labels)
annoy_index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


In [28]:
annoy_index.query(data_vectors[167])

[5, 5, 5, 5, 5, 5, 5]

Hierarchical Navigable Small World

In [29]:
!pip install nmslib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.5 MB 5.1 MB/s 
     |████████████████████████████████| 188 kB 47.7 MB/s 


In [30]:
import nmslib

In [31]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [32]:
hnsw_index = NMSLIBIndex(data_vectors, data_labels)
hnsw_index.build()

In [33]:
hnsw_index.query(data_vectors[1])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]